In [1]:
import pandas as pd

In [3]:
import pickle

In [6]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [44]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

In [150]:
from copy import deepcopy

In [64]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/johnrick/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Clean

In [46]:
df = pd.read_csv('top_5_comments_cleaned.csv')

In [51]:
df.drop(columns='Unnamed: 0', inplace=True)

In [54]:
# split comments and get rid of details (name, comment number, location, and time)
commentdf1= []
for i,row in df['comments'].str.split('\n', expand=True).iterrows():
    comments = ''
    try:
        if i != None:
            for j in row:
                if j != None:
                    if len(j.split()) >=10:
                        comments += j
    except:
        comments += 'missing comments'
    commentdf1.append(comments)

In [56]:
#create a column with the list of 10 last comments
df['comments_all'] = commentdf1

In [87]:
# get rid of all punctuation marks
df['comments_test'] = df['comments_all'].str.replace(r'[^\w\s]+', ' ')

In [88]:
# make all words lowercase
comments_real2 = []
for i in df['comments_test']:
    comments_real2.append(i.lower())

In [89]:
df['comments_test'] = comments_real2 

In [45]:
# stop words are common words that don't add much to details
stop_words =set(stopwords.words('english'))

In [101]:
len(stop_words)

179

In [114]:
type(stop_words)

set

In [115]:
add_stop_words = []

In [116]:
for i in list(stop_words):
    if "'" in i:
        add_stop_words.append(i.split("'")[0])
        add_stop_words.append(i.split("'")[1])

In [120]:
add_stop_words = set(add_stop_words)

In [124]:
len(stop_words)

179

In [122]:
len(stop_words | add_stop_words)

179

In [126]:
len(add_stop_words)

29

In [127]:
stop_words.update(add_stop_words)

In [132]:
add_stop_words = list(add_stop_words)

In [136]:
stop_words2 = set(add_stop_words+list(stop_words))

## Eliminate Stop Words

In [ ]:
stop_words2

In [141]:
# remove stop words from comments

df['comments_test'] = df['comments_test'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words2)]))
                           

In [142]:
df.head()

,about,address,comments,name,no_things_to_do,num_reviews,sights_landmarks,summary_overview,comments_all,comments_test
0,"About\nThrough commemoration, exhibitions and ...","180 Greenwich St, World Trade Center, New York...","1 - 10 of 61,964 reviews\nexiledsuburban\nOtta...",The National 9/11 Memorial & Museum,"#2 of 1,225 things to do in New York City","93,212 reviews","Sights & Landmarks, Museums, More",4.5,I recently visited the 9/11 Memorial and Museu...,recently visited 9 11 memorial museum second t...
1,About\nAt New York City's most visited museum ...,"1000 5th Ave, New York City, NY 10028-0198","1 - 10 of 31,397 reviews\nGarethKZN\nCape Town...",The Metropolitan Museum of Art,"#4 of 1,225 things to do in New York City","53,995 reviews","Sights & Landmarks, Museums, More",5.0,"You can get lost in this building, and even wi...",get lost building even good map probably disco...
2,"About\nFor more than 150 years, visitors have ...","59th to 110th Street, Manhattan Borough, from ...","1 - 10 of 73,814 reviews\nnehar897\n112\nRevie...",Central Park,"#3 of 1,225 things to do in New York City","130,863 reviews",Features Animals,4.5,It's the most famous park in the city for a re...,famous park city reasonit impossible visit pla...
3,About\nThe world-famous Empire State Building ...,"20 West 34th Street, New York City, NY 10118-0114","1 - 10 of 57,837 reviews\nAnto-Maher39\n5\nRev...",Empire State Building,"#13 of 1,225 things to do in New York City","87,837 reviews","Sights & Landmarks, Points of Interest & Landm...",4.5,Review of: Empire State Building Tickets - Obs...,review empire state building tickets observato...
4,"About\nTop of the Rock Observation Deck, the n...","30 Rockfeller Plaza, Enter on West 50th Street...","1 - 10 of 48,183 reviews\njewellerylover\nWarr...",Top of the Rock,"#7 of 1,225 things to do in New York City","77,744 reviews","Sights & Landmarks, Lookouts, Observation Deck...",4.5,We went on a clear day and the views were amaz...,went clear day views amazing able use chart id...


## Cosine Similarity

In [144]:
# set name to the index
df.set_index('name', inplace=True)

In [145]:
# calculate cosine similarity between documents - we use tfidf
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 3), min_df=0, stop_words='english')

In [147]:
tfidf_matrix = tf.fit_transform(df['comments_test'])

In [148]:
similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

In [149]:
indices = pd.Series(df.index)

In [156]:
df['location_nyc'] = df['address'].apply(lambda x: 1 if 'New York' in x else 0)

## Recommendation System

In [214]:
def recommendations_for_london(name, n=10, similarity_matrix = similarity_matrix, df=df):
    """
    returns a list of activities that are similar to name
    
    name: The activity name (must match Tripadvisor name)
    n: Highest number of activities to return
    similarity_matrix: uses cosine similarity to find similar activities
    df: dataframe that matches similarity matrix
    
    """
    
    recommended_activities = []
    
    # getting the index of the activities that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(similarity_matrix[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar name except itself
    top_20_indexes = list(score_series.iloc[1:300].index)
    
    temp = deepcopy(df)
    results = []
    for i in top_20_indexes:
        if temp.iloc[i, :].location_nyc == 0:
            results.append(i)
    
    for i in results:
        recommended_activities.append(list(df.index)[i])
    
    if len(recommended_activities) > n:
        return recommended_activities[:n]        
        
    return recommended_activities

In [217]:
recommendations_for_london('The National 9/11 Memorial & Museum', 15)

['Garden Museum',
 'Imperial War Museums',
 'Museum of London',
 'Fashion and Textile Museum',
 'The Cinema Museum',
 'Natural History Museum',
 'Royal Air Force Museum London',
 'National Maritime Museum',
 'The Household Cavalry Museum',
 'The British Museum',
 'The Who Shop',
 'Science Museum',
 'The Guards Museum',
 'Museum of London Docklands',
 'Gunnersbury Park & Museum']

In [211]:
def recommendations_for_nyc(name, n = 10, similarity_matrix = similarity_matrix, df=df):
    """
    returns a list of activities that are similar to name
    
    name: The activity name (must match Tripadvisor name)
    n: Highest number of activities to return
    similarity_matrix: uses cosine similarity to find similar activities
    df: dataframe that matches similarity matrix
    
    """
    
    recommended_activities = []
    
    # getting the index of the activities that matches the name
    idx = indices[indices == name].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(similarity_matrix[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar name except itself
    top_20_indexes = list(score_series.iloc[1:300].index)
    
    temp = deepcopy(df)
    results = []
    for i in top_20_indexes:
        if temp.iloc[i, :].location_nyc == 1:
            results.append(i)
    
    for i in results:
        recommended_activities.append(list(df.index)[i])
    
    if len(recommended_activities) > n:
        return recommended_activities[:n]    
        
    return recommended_activities

In [213]:
recommendations_for_nyc('Garden Museum', 15)

['New Museum of Contemporary Art',
 'American Folk Art Museum',
 'Museum of Arts and Design',
 'Cooper Hewitt, Smithsonian Design Museum',
 'Rubin Museum of Art',
 'Skyscraper Museum',
 'New York City Fire Museum',
 'Solomon R. Guggenheim Museum',
 'Ground Zero Museum Workshop',
 'The Metropolitan Museum of Art',
 'National Museum of the American Indian',
 'Whitney Museum of American Art',
 'African Burial Ground National Monument',
 'The National 9/11 Memorial & Museum',
 'South Street Seaport Museum']

In [184]:
nyc_activities_list = df[df['location_nyc']==1].index.to_list()

In [181]:
london_activities_list = df[df['location_nyc']==0].index.to_list()

In [187]:
print(random.choice(nyc_activities_list))

The New York Mosque
